# Manipulación Avanzada de Datos
------------------------

En esta sección aprenderemos a manipular nuestro dataframe haciendo agrupaciones de datos o trabajando con más de un dataframe a la vez


## 1. Sumarización de Datos

Las sentencias de agrupamiento de datos nos ayudan a brindar información resumida que pueda ser facilmente analizada por diversas personas.

<img src='./img/group_by.jpg'>

El agrupamiento de datos implica utilizar funciones de agregacion como: `count`, `sum`, `mean`, `min`, `max` a una columna del df

In [ ]:
import pandas as pd

df_reviews = pd.read_csv('./src/winemag-data-130k-v2.csv')
df_reviews.head(2)

In [ ]:

# groupby -> lista de columnas a agrupar
df_reviews.groupby(['country']).price.agg([len, 'min', 'max', 'mean'])

In [ ]:
df_group = df_reviews.groupby(['country', 'province']).agg(
    # columna: metosdos agregacion a aplicar
    {'points': ['mean', 'min', 'max'],
     'price': ['mean', 'min', 'max']
    }
    # ordenando por points descendentemente y price ascendentemente
).sort_values(by=[('points', 'mean'), ('price', 'mean')], ascending=[False, True])

# mostrando top 5
df_group.head(5)

## 2. Manipulando más de un DataFrame

### 2.1 Unificando o Concatenando DataFrames

Esto nos permite unificar información de Dataframs cuyas columnas sean iguales

<img src='https://pandas.pydata.org/docs/_images/08_concat_row.svg'>

In [ ]:
# A manera de ejemplo veremos la unificación de 2 df's cuya data es similar

import pandas as pd

columns = ["date.utc", "location", "parameter", "value"]

df_air_quality_no2 = pd.read_csv("./src/air_quality_no2_long.csv", parse_dates=True, usecols=columns)
df_air_quality_pm25 = pd.read_csv("./src/air_quality_pm25_long.csv", parse_dates=True, usecols=columns)


In [ ]:
df_air_quality_no2.head(2)



In [ ]:
df_air_quality_pm25.head()

In [ ]:
# Unificando la información en un único df

df_air_quality = pd.concat([df_air_quality_pm25, df_air_quality_no2], axis=0)
df_air_quality.head()

In [ ]:
df_air_quality

In [ ]:
# corrección del índice
df_air_quality = df_air_quality.reset_index(drop=True)
df_air_quality

In [ ]:
print('Shape of the ``air_quality_pm25`` table: ', df_air_quality_pm25.shape)

print('Shape of the ``air_quality_no2`` table: ', df_air_quality_no2.shape)

print('Shape of the resulting ``air_quality`` table: ', df_air_quality.shape)

### 2.2 Joins

Podemos unir dos dataframes en funcion de sus columnas comunes usando `merge`

La operacion merge implica combinar 2 df a partir de uno o más valores llave o `key`

<img src='./img/merge.png'>

In [ ]:
# Unificaremos la información consolidada del df previo 
df_stations_coord = pd.read_csv("./src/air_quality_stations.csv")
df_stations_coord.head(2)

In [ ]:
df_stations_coord.shape

In [ ]:
# Nos damos cuenta que existen valores duplicados sobre el campo llave de 'location'
df_stations_coord.drop_duplicates(subset=['location']).shape

In [ ]:
df_stations_coord_no_duplicates = df_stations_coord.drop_duplicates(subset=['location'], keep='first')
df_stations_coord_no_duplicates.reset_index(inplace=True,drop=True)
df_stations_coord_no_duplicates.head(2)

In [ ]:
# como llame emplearemos la columna 'location'

df_air_quality = pd.merge(df_air_quality, df_stations_coord_no_duplicates, how="inner", on="location")
df_air_quality.head()

In [ ]:
# Notamos un incremento de los registros, luego del inner join
df_air_quality.shape

In [ ]:
# como llave emplearemos la columna 'location'
df_air_quality_2 = pd.merge(df_air_quality, df_stations_coord_no_duplicates, how="inner", left_on='location', right_on='location')

df_air_quality_2.head()

Como punto general existen diferentes formas de combinar los dataframe, siendo el método `inner` el utilizado por defecto

<img src='./img/merge_tipos.png'>

## Información Adicional


- Group By Explicado:  https://learnsql.com/blog/group-by-in-sql-explained/

- Combinando Múltiples dataFrames : https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html#min-tut-08-combine

In [ ]:
# Resolviendo PC4 - Ejercicio 5 del examen.
# Empleando Pandas

from pymongo import MongoClient
from pymongoarrow.api import Schema
from pymongoarrow.monkey import patch_all
import pyarrow as pa
import pandas as pd

RUTA_ARCHIVO = '/workspaces/ProgramacionPython202506/Modulo4/src/ventas.csv'
# Configura los detalles de conexión
RUTA_CADENA_CONEXION = "/workspaces/ProgramacionPython202506/Modulo4/scripts/problema4/cadena.txt"
mongo_uri = open(RUTA_CADENA_CONEXION).read().strip()

# Aplica el monkey patch para usar pymongoarrow
patch_all()

In [ ]:

nombres_columnas = ['fecha_venta', 'product_name', 'cantidad', 'precio']
df_ventas =  pd.read_csv(RUTA_ARCHIVO, sep=',', header=None)

# renombrando columnas
df_ventas.columns = nombres_columnas

df_ventas

In [ ]:
df_ventas.dtypes

In [ ]:
# Traer informacion de TC de MongoDB

with MongoClient(mongo_uri) as client:
    # Selecciona la base de datos y la colección
    db = client['curso_db']
    collection = db['sunat']

    # Define el esquema para los datos que quieres leer
    # schema = Schema({"campo1": pa.string(), "campo2": pa.int64(), "campo3": pa.float64()})

    # Lee los datos desde MongoDB en un DataFrame de pandas usando pymongoarrow
    df_tc = collection.find_pandas_all({})
    pass

df_tc.tail()

In [ ]:
df_tc.shape

In [ ]:
df_tc.dtypes

In [ ]:
# Realizo operacion de merge de los df 

df_ventas_consolidado = pd.merge(df_ventas, df_tc, how='left', left_on='fecha_venta', right_on='fecha')

df_ventas_consolidado

In [ ]:
# eliminamos columnas que no son de interes

df_ventas_consolidado.drop(columns=['_id','origen','moneda','fecha'], inplace=True, axis=0)
df_ventas_consolidado


In [ ]:
df_ventas_consolidado['precio_solarizado'] = df_ventas_consolidado.precio * df_ventas_consolidado.venta
df_ventas_consolidado['subtotal'] = df_ventas_consolidado.precio_solarizado * df_ventas_consolidado.cantidad
df_ventas_consolidado

In [ ]:
df_agrupago = df_ventas_consolidado.groupby(['product_name']).agg(
    {'subtotal': 'sum'}
)
df_agrupago